In [121]:
import pandas as pd

# Refining

In [122]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
EFF_H2_SMR = 159.6 #MJ/kgH2

In [123]:
lcoh_res_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx')
res_be_df = pd.DataFrame(columns=['RES', 'Breakeven price ($/MMBtu)'])
res_be_df['RES'] = lcoh_res_df['RES']
res_be_df['Breakeven price ($/MMBtu)'] = lcoh_res_df.apply(lambda x:x['LCOH']/(EFF_H2_SMR*CONV_MJ_TO_MMBTU), axis=1)
res_be_df

,RES,Breakeven price ($/MMBtu)
0,Grid PEM,34.837997
1,Wind subsidized,37.250875
2,Wind unsubsidized,43.861501
3,Solar PEC,49.579692
4,Solar PV-E,50.901817
5,SMR 89% CCUS,12.758507


In [124]:
res_be_df.to_csv('./results/res_be_refining.csv', header=True, index=False)


## Steel

In [125]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
COAL_CONS_RATE = 0.663 #ton-coal/ton-steel for conventional BF/BOF plant
iron_ore_cost = 100 #$/t_ironore
bfbof_iron_cons = 1.226 #t_ironore/t_steel
om_bfbof = 353.25 #$/t_steel
DRICO2Intensity = 40 # kgCO2/ton-DRI`````````````````
ShaftFCAPEX = 250 # $/tDRI/year
EAFCAPEX = 160 # $/tsteel/year
EAFOM  = 24.89 # $/tsteel (EAF and casting)
Steel = 800 # $/tsteel
RatioSteelDRI = 0.9311 # tsteel/tDRI
RatioIronOreDRI = 1.391 # tironore/tDRI


In [126]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [127]:
steel_df = pd.read_excel('h2_demand_bfbof_steel_us_2022.xlsx', sheet_name='processed')
steel_ids = list(steel_df['Plant'])

In [128]:
res_be_df= pd.DataFrame(columns=['Plant', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['Plant'] = steel_df['Plant']
res_be_df.set_index('Plant', inplace=True)


In [129]:
from opt_deployment_steel import get_steel_plant_demand
def compute_res_be(lcoh, lcoe, plant):
  steel_cap_ton_per_annum, h2_dem_kg_per_day, elec_dem_MWe = get_steel_plant_demand(plant)
  costs_dri_eaf = steel_cap_ton_per_annum*(EAFCAPEX + ShaftFCAPEX/RatioSteelDRI + EAFOM +\
            iron_ore_cost*RatioIronOreDRI/RatioSteelDRI)
  costs_h2 =  lcoh*h2_dem_kg_per_day*365
  costs_electricity = lcoe*elec_dem_MWe*24*365
  costs_bfbof = iron_ore_cost*bfbof_iron_cons*steel_cap_ton_per_annum + om_bfbof*steel_cap_ton_per_annum
  coal_ann_consumption = steel_cap_ton_per_annum*COAL_CONS_RATE
  res_be = (costs_h2+costs_electricity+costs_dri_eaf - costs_bfbof)/coal_ann_consumption
  return res_be

In [130]:
for plant in steel_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be(lcoh, lcoe, plant)
    res_be_df.loc[plant, res] = res_be

res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/tcoal)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_steel.csv', header=True, index=False)

# Heat Process